In [98]:
import os
import pandas as pd
import numpy as np

pd.set_option('mode.chained_assignment', None)

In [99]:
df = pd.read_csv('제주특별자치도_태양광발전소현황_20240630.csv', encoding = 'cp949')
df.head(1)

,NO,허가일자,상호,설비용량(KW),설치장소,상태,사업개시일,데이터기준일자
0,1,2006-03-03,오복산전,59.87,제주특별자치도 제주시 한경면 신창리 760번지 2호,사업개시,2006-07-11,2024-06-30


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1643 entries, 0 to 1642
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NO        1643 non-null   int64  
 1   허가일자      1643 non-null   object 
 2   상호        1643 non-null   object 
 3   설비용량(KW)  1643 non-null   float64
 4   설치장소      1643 non-null   object 
 5   상태        1643 non-null   object 
 6   사업개시일     1643 non-null   object 
 7   데이터기준일자   1643 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 102.8+ KB


In [101]:
#사업개시일이 있는 데이터만 골라낸다.
#사업개시일이 없는 데이터는 아직 인허가가 진행중이므로 drop한다.
df = df.dropna(axis = 0)
df

,NO,허가일자,상호,설비용량(KW),설치장소,상태,사업개시일,데이터기준일자
0,1,2006-03-03,오복산전,59.87,제주특별자치도 제주시 한경면 신창리 760번지 2호,사업개시,2006-07-11,2024-06-30
1,2,2006-08-24,오복산전 3호,29.52,제주특별자치도 제주시 한경면 신창리 760번지 3호,사업개시,2006-09-25,2024-06-30
2,3,2007-01-05,민솔라에너지,90.72,제주특별자치도 제주시 조천읍 대흘리 110번지 4호,사업개시,2007-04-04,2024-06-30
3,4,2007-02-21,(주)도암엔지니어링건축사사무소,3.00,제주특별자치도 제주시 한경면 일주서로 4716-8,사업개시,2007-02-27,2024-06-30
4,5,2007-02-21,제주에너지 주식회사,200.00,제주특별자치도 서귀포시 성산읍 신산리 1960번지,사업개시,2008-09-29,2024-06-30
...,...,...,...,...,...,...,...,...
1638,1639,2021-04-30,가시일 태양광발전소,398.39,제주특별자치도 서귀포시 표선면 가시리 677번지,사업개시,2023-04-27,2024-06-30
1639,1640,2021-08-03,태양7호태양광발전소,198.88,제주특별자치도 서귀포시 안덕면 상천리 764-4,사업개시,2021-12-10,2024-06-30
1640,1641,2021-08-03,종달2호태양광발전소,99.00,제주특별자치도 제주시 구좌읍 종달리 3785번지 .3787.3785-3,사업개시,2023-12-26,2024-06-30
1641,1642,2021-09-15,흥산태양광발전소,999.63,제주특별자치도 제주시 구좌읍 행원리 2686번지 .2694,사업개시,2022-04-25,2024-06-30


In [102]:
# 사업개시일이 기록되었는데 상태가 '공사진행', '인허가'인 경우
df[df['상태'] != '사업개시']
#모든 상태가 사업개시인 상태이므로 추가 수정 불필요

,NO,허가일자,상호,설비용량(KW),설치장소,상태,사업개시일,데이터기준일자


In [103]:
#사업개시일을 시간순으로 정렬하기 위해 문자열타입에서 datetime타입으로 변경해준다.
df['사업개시일'] = pd.to_datetime(df['사업개시일'])
#sort_values함수를 이용해 사업개시일 컬럼값을 기준으로 정렬한다.
df.sort_values(by = '사업개시일', inplace = True)

#데이터프레임의 인데스를 새롭게 생성한다.
df.reset_index(drop = True, inplace = True)

In [104]:
# 파일명에는 태양광 발전소 현황이지만 태양광만이 아닌 다른 에너지 발전소 전체의 정보가 포함되어 있음
# 상호만으로 태양광 정보를 걸러내기에는 무리가 있어보임.
# 원동력종류가 표기된 파일을 추가로 다운받는다.

df2 = pd.read_csv('제주특별자치도_신재생에너지발전시설현황_20221116.csv', encoding = 'cp949')
df2['원동력종류']

0          풍력
1          풍력
2          풍력
3          풍력
4          풍력
        ...  
1577    바이오가스
1578    바이오가스
1579    바이오가스
1580    바이오가스
1581       기타
Name: 원동력종류, Length: 1582, dtype: object

In [74]:
#원동력 종류에서 태양광만 추출하기
df2 = df2[df2['원동력종류'] == '태양광']
df2.reset_index(drop = True, inplace = True)
df2

,허가일자,상호,설비용량(KW),설치장소,원동력종류,사업개시일,데이터기준일자
0,2021-09-15,흥산태양광발전소,999.63,제주특별자치도 제주시 구좌읍,태양광,2022-04-25,2022-11-16
1,2021-08-03,태양7호태양광발전소,198.88,제주특별자치도 서귀포시 안덕면,태양광,2021-12-10,2022-11-16
2,2021-04-21,제주덕암 태양광발전소,99.18,제주특별자치도 서귀포시 표선면,태양광,2022-01-03,2022-11-16
3,2021-04-19,청춘 태양광발전소,662.48,제주특별자치도 서귀포시 안덕면,태양광,2022-01-03,2022-11-16
4,2021-04-19,에스원태양광발전소,458.64,제주특별자치도 서귀포시 안덕면,태양광,2022-01-03,2022-11-16
...,...,...,...,...,...,...,...
1542,2007-02-21,제주에너지 주식회사,200.00,제주특별자치도 서귀포시 성산읍,태양광,2008-09-29,2022-11-16
1543,2007-02-21,제주에너지 주식회사,200.00,제주특별자치도 제주시 애월읍,태양광,2008-04-18,2022-11-16
1544,2007-01-05,민솔라에너지,90.72,제주특별자치도 제주시 조천읍,태양광,2007-04-04,2022-11-16
1545,2006-08-24,오복산전 3호,29.52,제주특별자치도 제주시 한경면,태양광,2006-09-25,2022-11-16


In [75]:
#주소가 중복되는 경우 하나만 출력하기 위해 unique()함수를 사용합니다.
geo_addr = df2['설치장소'].unique()

geo_pos = []

#위도와 경도로 변환시 주소에 '시'가 포함되어 있으면 정상작동 안하기에 '시'를 공백으로대체
for addr in geo_addr:
    geo_pos.append(addr.replace('시',''))

print(geo_pos)

['제주특별자치도 제주 구좌읍', '제주특별자치도 서귀포 안덕면', '제주특별자치도 서귀포 표선면', '제주특별자치도 서귀포 성산읍', '제주특별자치도 서귀포 법환동', '제주특별자치도 서귀포 대정읍', '제주특별자치도 제주 한림읍', '제주특별자치도 서귀포 남원읍', '제주특별자치도 서귀포 상예동', '제주특별자치도 서귀포 회수동', '제주특별자치도 제주 조천읍', '제주특별자치도 제주 도남동', '제주특별자치도 제주 애월읍', '제주특별자치도 서귀포 토평동', '제주특별자치도 서귀포 강정동', '제주특별자치도 서귀포 중문동', '제주특별자치도 제주 봉개동', '제주특별자치도 서귀포 월평동', '제주특별자치도 제주 용담이동', '제주특별자치도 제주 일도이동', '제주특별자치도 제주 회천동', '제주특별자치도 제주 해안동', '제주특별자치도 서귀포 호근동', '제주특별자치도 제주 오라삼동', '제주특별자치도 제주 한경면', '제주특별자치도 제주 아라이동', '제주특별자치도 제주 오라이동', '제주특별자치도 제주 용강동', '제주특별자치도 제주 외도일동', '제주특별자치도 제주 삼양일동', '제주특별자치도 서귀포 도순동', '제주특별자치도 서귀포 상효동', '제주특별자치도 서귀포 서홍동', '제주특별자치도 제주 이도일동', '제주특별자치도 제주 오등동', '제주특별자치도 서귀포 서호동', '제주특별자치도 제주 영평동', '제주특별자치도 서귀포 동홍동', '제주특별자치도 서귀포 하원동', '제주특별자치도 제주 이호이동', '제주특별자치도 제주 화북이동', '제주특별자치도 서귀포 색달동', '제주특별자치도 서귀포 서귀동', '제주특별자치도 제주 도련이동', '제주특별자치도 제주 오라일동', '제주특별자치도 서귀포 하효동', '제주특별자치도 제주 도평동', '제주특별자치도 서귀포 하예동', '제주특별자치도 제주 노형동', '제주특별자치도 제주 삼도이동', '제주특별자치도 제주 도련일동', '제주특별자치도 제주 아라일동']


In [112]:
#주소값을 geopy패키지를 활용하여 경도와 위도 값을 얻는 함수를 만든다.
from geopy.geocoders import Nominatim

unique_addr = []
unique_lat = []
unique_lng = []

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South korea', timeout = None)
    geo = geolocoder.geocode(address)
    if geo == None:
        print('geo is null')
        return 0, 0
    else:
        return geo.latitude, geo.longitude


In [113]:
for addr in geo_pos:
    print(addr)
    lat, lng = geocoding(addr)
    if lat == 0 or lng == 0:
        continue
    unique_addr.append(addr)
    unique_lat.append(lat)
    unique_lng.append(lng)
    print(lat, lng)

제주특별자치도 제주 구좌읍
33.51662 126.81055
제주특별자치도 서귀포 안덕면
33.27688 126.34925
제주특별자치도 서귀포 표선면
33.3504543 126.794049
제주특별자치도 서귀포 성산읍
33.41805 126.88147
제주특별자치도 서귀포 법환동
33.24113 126.51285
제주특별자치도 서귀포 대정읍
33.2218157 126.2529945
제주특별자치도 제주 한림읍
33.39344 126.26684
제주특별자치도 서귀포 남원읍
33.3076683 126.6907475
제주특별자치도 서귀포 상예동
33.26177 126.38678
제주특별자치도 서귀포 회수동
33.26549 126.44363
제주특별자치도 제주 조천읍
33.50888 126.66305
제주특별자치도 제주 도남동
33.4905148 126.5239964
제주특별자치도 제주 애월읍
33.45079 126.3752
제주특별자치도 서귀포 토평동
33.2705286 126.5841149
제주특별자치도 서귀포 강정동
33.24695 126.48927
제주특별자치도 서귀포 중문동
33.262002 126.4288617
제주특별자치도 제주 봉개동
33.484188 126.5993497
제주특별자치도 서귀포 월평동
33.24501 126.46294
제주특별자치도 제주 용담이동
33.50742 126.50615
제주특별자치도 제주 일도이동
33.50864 126.53795
제주특별자치도 제주 회천동
33.4956 126.61479
제주특별자치도 제주 해안동
33.44899 126.46036
제주특별자치도 서귀포 호근동
33.25632 126.53351
제주특별자치도 제주 오라삼동
33.49487 126.50508
제주특별자치도 제주 한경면
33.3305 126.21066
제주특별자치도 제주 아라이동
33.48148 126.55055
제주특별자치도 제주 오라이동
33.47081 126.50991
제주특별자치도 제주 용강동
33.47017 126.58934
제주특별자치도 

In [114]:
df_unique = pd.DataFrame({'주소':unique_addr,
                          '경도(x)':unique_lng,
                          '위도(y)':unique_lat})
df_unique.head()



,주소,경도(x),위도(y)
0,제주특별자치도 제주 구좌읍,126.810550,33.516620
1,제주특별자치도 서귀포 안덕면,126.349250,33.276880
2,제주특별자치도 서귀포 표선면,126.794049,33.350454
3,제주특별자치도 서귀포 성산읍,126.881470,33.418050
4,제주특별자치도 서귀포 법환동,126.512850,33.241130


In [116]:
#Folium을 사용하여 marker 표시하기
import pandas as pd
import folium




jeju_map = folium.Map(location=[33.38, 126.55], zoom_start=11)

for i in range(len(df_unique)):
    pos_list = [df_unique.iloc[i]['위도(y)'], df_unique.iloc[i]['경도(x)']]
    folium.Marker(pos_list, popup=df_unique.iloc[i]['주소']).add_to(jeju_map)
    
jeju_map.save('jeju.html')
